# Analise de clientes inadimplentes 

Este projeto tem como objetivo analisar os dados de crédito de uma instituição financeira, focando principalmente na variável "default". 
Essa variável indica se um cliente é adimplente (default = 0) ou inadimplente (default = 1). O objetivo é compreender os motivos pelos 
quais os clientes deixam de honrar suas dívidas, considerando o comportamento em relação a outros atributos, tais como salário, escolaridade e movimentação financeira.

Descrição dos Atributos:
| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |


Com base nos dados disponíveis, vamos explorar as seguintes questões:

* Qual é a taxa geral de inadimplência na instituição financeira?
* Qual é a distribuição dos clientes em relação aos níveis de escolaridade?
* Existe alguma relação entre a escolaridade e a taxa de inadimplência?
* Como a movimentação financeira impacta a probabilidade de inadimplência?
* Existe uma relação entre o salário e a probabilidade de inadimplência?

Para responder às questões propostas, seguiremos os seguintes passos:

1. Pré-processamento dos dados: Verificaremos se há valores ausentes, inconsistências ou ruídos nos dados. Realizaremos a limpeza e o tratamento necessário.

2. Análise exploratória: Realizaremos uma análise descritiva dos dados, obtendo estatísticas resumidas e visualizando a distribuição dos atributos relevantes.

3. Análise de correlação: Investigaremos a relação entre os atributos selecionados e a variável "default" por meio de análises estatísticas e gráficos.

4. Modelagem preditiva: Utilizaremos técnicas de aprendizado de máquina, como regressão logística ou árvores de decisão, para construir um modelo preditivo capaz de prever a probabilidade de inadimplência com base nos atributos selecionados.

5. Avaliação do modelo: Avaliaremos a performance do modelo utilizando métricas apropriadas, como acurácia, precisão, recall e curva ROC.



Com base na análise realizada, poderemos identificar os principais fatores que contribuem para a inadimplência dos clientes na instituição financeira. Essas informações serão valiosas para a tomada de decisões estratégicas, como ações de recuperação de crédito e análise de risco de novos clientes.

## 1. Pré-processamento dos dados

In [6]:
#importando as bibliotecas e dataframe
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df_clientes = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRSP1vWDhRIr8b-liTIHKueclNC7LyS0WFpPO9E5X4oztE9F2ugzkj_UfEYoIf_YCJlK13Xmkc__siZ/pub?output=csv', na_values='na')

In [7]:
df_clientes.head(10)

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,818770008,0,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,713982108,0,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,769911858,0,40,F,4,ensino medio,NaN,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,709106358,0,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28
5,713061558,0,44,M,2,mestrado,casado,$40K - $60K,blue,36,3,2,1,"4.010,69","1.088,07",24
6,810347208,0,51,M,4,NaN,casado,$120K +,gold,46,6,3,1,"34.516,72","1.330,87",31
7,818906208,0,32,M,0,ensino medio,NaN,$60K - $80K,silver,27,2,2,2,"29.081,49","1.538,32",36
8,710930508,0,37,M,3,sem educacao formal,solteiro,$60K - $80K,blue,36,5,0,2,"22.352,50","1.350,14",24
9,719661558,0,48,M,2,mestrado,solteiro,$80K - $120K,blue,36,6,3,3,"11.656,41","1.441,73",32


### 1.1. Estrutura

#### Analisando a proporção de clientes adimplentes e inadimplentes

In [8]:
df_clientes.shape # retorna uma tupla (qtd linhas, qtd colunas)

(10127, 16)

In [10]:
df_clientes[df_clientes['default'] == 0].shape

(8500, 16)

In [12]:
df_clientes[df_clientes['default'] == 1].shape

(1627, 16)

In [17]:
qtd_total, _ = df_clientes.shape
qtd_adimplentes, _ = df_clientes[df_clientes['default'] == 0].shape
qtd_inadimplentes, _ = df_clientes[df_clientes['default'] == 1].shape

In [19]:
print(f" A proporção de clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f" A proporção de clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

 A proporção de clientes adimplentes é de 83.93%
 A proporção de clientes inadimplentes é de 16.07%


### 1.2. Schema

In [20]:
df_clientes.head(n=5)

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,818770008,0,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,713982108,0,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,769911858,0,40,F,4,ensino medio,NaN,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,709106358,0,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28


In [21]:
df_clientes.tail(n=5)

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
10122,772366833,0,50,M,2,mestrado,solteiro,$40K - $60K,blue,40,3,3,2,"4.003,91","15.476,26",117
10123,710638233,1,41,M,2,NaN,divorciado,$40K - $60K,blue,25,4,3,2,"4.277,04","8.764,88",69
10124,716506083,1,44,F,1,ensino medio,casado,menos que $40K,blue,36,5,4,3,"5.409,16","10.291,78",60
10125,717406983,1,30,M,2,mestrado,NaN,$40K - $60K,blue,36,4,3,3,"5.281,84","8.395,62",62
10126,714337233,1,43,F,2,mestrado,casado,menos que $40K,silver,25,6,4,2,"10.388,80","10.294,96",61


* Colunas e seus respectivos tipos de dados. (note que as colunas **limite_credito** e **valor_transacoes_12m** estão como `dtype = object`)

In [22]:
df_clientes.dtypes

id                          int64
default                     int64
idade                       int64
sexo                       object
dependentes                 int64
escolaridade               object
estado_civil               object
salario_anual              object
tipo_cartao                object
meses_de_relacionamento     int64
qtd_produtos                int64
iteracoes_12m               int64
meses_inativo_12m           int64
limite_credito             object
valor_transacoes_12m       object
qtd_transacoes_12m          int64
dtype: object

* Atributos categóricos.

In [24]:
df_clientes.select_dtypes('object').describe().transpose() 

,count,unique,top,freq
sexo,10127,2,F,5358
escolaridade,8608,5,mestrado,3128
estado_civil,9378,3,casado,4687
salario_anual,9015,5,menos que $40K,3561
tipo_cartao,10127,4,blue,9436
limite_credito,10127,9272,"1.438,21",11
valor_transacoes_12m,10127,10035,"3.851,51",3


* Atributos numéricos

In [26]:
df_clientes.drop('id', axis=1).select_dtypes('number').describe().transpose() 

,count,mean,std,min,25%,50%,75%,max
default,10127.0,0.160660,0.367235,0.0,0.0,0.0,0.0,1.0
idade,10127.0,46.325960,8.016814,26.0,41.0,46.0,52.0,73.0
dependentes,10127.0,2.346203,1.298908,0.0,1.0,2.0,3.0,5.0
meses_de_relacionamento,10127.0,35.928409,7.986416,13.0,31.0,36.0,40.0,56.0
qtd_produtos,10127.0,3.812580,1.554408,1.0,3.0,4.0,5.0,6.0
iteracoes_12m,10127.0,2.455317,1.106225,0.0,2.0,2.0,3.0,6.0
meses_inativo_12m,10127.0,2.341167,1.010622,0.0,2.0,2.0,3.0,6.0
qtd_transacoes_12m,10127.0,64.858695,23.472570,10.0,45.0,67.0,81.0,139.0


### 1.3 Dados faltantes

In [27]:
df_clientes.head()

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,818770008,0,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,713982108,0,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,769911858,0,40,F,4,ensino medio,NaN,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,709106358,0,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28


* Verificando dados faltantes 

In [30]:
df_clientes.isna().any()

id                         False
default                    False
idade                      False
sexo                       False
dependentes                False
escolaridade                True
estado_civil                True
salario_anual               True
tipo_cartao                False
meses_de_relacionamento    False
qtd_produtos               False
iteracoes_12m              False
meses_inativo_12m          False
limite_credito             False
valor_transacoes_12m       False
qtd_transacoes_12m         False
dtype: bool

* A função abaixo levanta algumas estatisticas sobre as colunas dos dados faltantes.

In [35]:
def stats_dados_faltantes(df_clientes: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df_clientes.columns:
    if df_clientes[col].isna().any():
      qtd, _ = df_clientes[df_clientes[col].isna()].shape
      total, _ = df_clientes.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [36]:
stats_dados_faltantes(df_clientes=df_clientes)

{'escolaridade': {'quantidade': 1519, 'porcentagem': 15.0}}
{'estado_civil': {'quantidade': 749, 'porcentagem': 7.4}}
{'salario_anual': {'quantidade': 1112, 'porcentagem': 10.98}}


In [37]:
stats_dados_faltantes(df_clientes=df_clientes[df_clientes['default'] == 0])

{'escolaridade': {'quantidade': 1263, 'porcentagem': 14.86}}
{'estado_civil': {'quantidade': 620, 'porcentagem': 7.29}}
{'salario_anual': {'quantidade': 925, 'porcentagem': 10.88}}


In [38]:
stats_dados_faltantes(df_clientes=df_clientes[df_clientes['default'] == 1])

{'escolaridade': {'quantidade': 256, 'porcentagem': 15.73}}
{'estado_civil': {'quantidade': 129, 'porcentagem': 7.93}}
{'salario_anual': {'quantidade': 187, 'porcentagem': 11.49}}


### 1.4 Correção de schema

* Na etapa de exploração, foi identificado que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas `dtype = object`.


In [40]:
df_clientes[['limite_credito', 'valor_transacoes_12m']].dtypes

limite_credito          object
valor_transacoes_12m    object
dtype: object

In [41]:
df_clientes[['limite_credito', 'valor_transacoes_12m']].head()

,limite_credito,valor_transacoes_12m
0,"12.691,51","1.144,90"
1,"8.256,96","1.291,45"
2,"3.418,56","1.887,72"
3,"3.313,03","1.171,56"
4,"4.716,22","816,08"


* Criando uma função `lambda` para limpar os dados, convertendo valores no formato de string com vírgula (representando separador decimal) para o formato float com ponto decimal, em seguida testando sua aplicação através do método funcional `map`

In [43]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
[12691.51, 8256.96, 3418.56, 3313.03, 4716.22]


* Aplicando a função lambda nas colunas de interesse 

In [ ]:
df_clientes['valor_transacoes_12m'] = df_clientes['valor_transacoes_12m'].apply(fn)
df_clientes['limite_credito'] = df_clientes['limite_credito'].apply(fn)